In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
from bs4 import BeautifulSoup as BS
import re

# df_search = pd.read_csv(r"C:\Users\panka\OneDrive\Desktop\Karnataka Scraped Data\Google Map\Google MAp Scraped Data\Restaurants\Homestays to search.csv")

In [2]:
##  -- Here

nme = "Places to visit in Delhi"
nme

'Places to visit in Delhi'

In [3]:
##  -- Here

zz = "Delhi"

In [4]:
url = "https://www.google.com/maps/@25.1297792,75.8448128,14z?entry=ttu"


driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

time.sleep(1.5)

wait = WebDriverWait(driver,10)
search_input = nme
input_area = wait.until(EC.presence_of_element_located((By.XPATH, '//input[@id="searchboxinput"]')))
input_area.clear()
input_area.send_keys(search_input)

search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@id="searchbox-searchbutton"]')))
search_button.click()

In [5]:
# time.sleep(4)

divSideBar=driver.find_element(By.CSS_SELECTOR,f"div[aria-label='Results for {search_input}']")
keepScrolling=True
while(keepScrolling):
    divSideBar.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)
    divSideBar.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)
    html =driver.find_element(By.TAG_NAME, "html").get_attribute('outerHTML')
    if(html.find("You've reached the end of the list.")!=-1):
        keepScrolling=False

In [6]:
list_data = driver.find_elements(By.XPATH, f'//div[@aria-label="Results for {search_input}"]/div')

Restaurants_Links = []

for j in list_data:
    try:
        Restaurants_Links.append(j.find_element(by="xpath", value='.//div/a').get_attribute("href"))
    except NoSuchElementException:
        Restaurants_Links.append("None")

cleaned_restaurants_Links = [k for k in Restaurants_Links if k != "None"]

In [7]:
len(cleaned_restaurants_Links)

119

In [8]:
Name = []
Address =[]
Website = []
Phone_No =[]
Category = []
Rating = []
No_of_Reviews = []
linkk = []
Location_List = []

In [9]:
for i in cleaned_restaurants_Links:
    
    driver.get(i)
        
    try:
        Name.append(driver.find_element(by="xpath", value ='//h1[@class="DUwDvf lfPIob"]').text)
    except NoSuchElementException:
        Name.append("None")

    try:
        Address.append(driver.find_element(by="xpath", value='//button[contains(@aria-label, "Address")]').get_attribute("aria-label").lstrip("Address:"))
    except NoSuchElementException:
        Address.append("None")

    try:
        Website.append(driver.find_element(by="xpath", value='//a[contains(@aria-label, "Website")]/div/div[2]/div[1]').text)
    except NoSuchElementException:
        Website.append("None")

    try:
        Phone_No.append(driver.find_element(by="xpath", value='//button[contains(@aria-label, "Phone")]').get_attribute("aria-label").lstrip("Phone:"))
    except NoSuchElementException:
        Phone_No.append("None")

    try:
        Category.append(driver.find_element(by="xpath", value='//button[contains(@jsaction,"category")]').text)
    except NoSuchElementException:
        Category.append("None")

    try:
        Rating.append(driver.find_element(by="xpath", value='//span[contains(@aria-label,"stars ") and contains(@class,"ceNzKf")]').get_attribute("aria-label").replace("stars", "").strip())
    except NoSuchElementException:
        Rating.append("None")

    try:
        No_of_Reviews.append(driver.find_element(by="xpath", value='//span[contains(@aria-label, "reviews")]').text.strip("()"))
    except NoSuchElementException:
        No_of_Reviews.append("None")

    linkk.append(driver.current_url)

    Location_List.append(zz)

        
    

driver.quit()
        

In [10]:
df = pd.DataFrame({
    "Name": Name,
    "Address": Address,
    "Website": Website,
    "Phone No": Phone_No,
    "Category": Category,
    "Ratings": Rating,
    "Reviews": No_of_Reviews,
    "Google Map Link":linkk,
    "Location": Location_List
})

In [11]:
df

,Name,Address,Website,Phone No,Category,Ratings,Reviews,Google Map Link,Location
0,None,None,None,None,None,None,None,https://support.google.com/maps/answer/3092445...,Delhi
1,Jantar Mantar,"J6G8+RMC, Connaught Place, Sansad Marg, New D...",delhitourism.gov.in,None,Observatory,4.2,"30,499",https://www.google.com/maps/place/Jantar+Manta...,Delhi
2,India Gate,"Kartavya Path, India Gate, New Delhi, Delhi 1...",delhitourism.gov.in,011 2336 5358,Monument,4.6,"259,375",https://www.google.com/maps/place/India+Gate/d...,Delhi
3,"Purana Quila, Delhi","J65V+RFH, Mathura Rd, Near Delhi Zoo, New Del...",delhitourism.gov.in,011 2336 5358,Fortress,4.2,"30,695",https://www.google.com/maps/place/Purana+Quila...,Delhi
4,Lodhi Garden,"Lodhi Rd, Lodhi Gardens, Lodhi Estate, New De...",ndmc.gov.in,011 2336 5358,Garden,4.5,"46,648",https://www.google.com/maps/place/Lodhi+Garden...,Delhi
...,...,...,...,...,...,...,...,...,...
114,Buddha Garden,"M6Q6+RRW, Faculty of Science, University Encl...",None,011 2336 5358,Park,4.3,"7,005",https://www.google.com/maps/place/Buddha+Garde...,Delhi
115,Sahaj Anand- Multimedia Water Show- Akshardham,"J76H+P9P, Pandav Nagar, New Delhi, Delhi, 110...",akshardham.com,011 4344 2344,Tourist attraction,4.7,278,https://www.google.com/maps/place/Sahaj+Anand-...,Delhi
116,Snow World,"L05, Dlf Mall Of India, Sector 18, Noida, Utt...",snowworldindia.com,099306 44001,Indoor snowcenter,4.1,"10,897",https://www.google.com/maps/place/Snow+World/d...,Delhi
117,Worlds of Wonder,"No. 11, The Great India Place Mall, Sector-38...",worldsofwonder.in,080 6909 0000,Water park,4.3,"28,400",https://www.google.com/maps/place/Worlds+of+Wo...,Delhi


In [12]:
df.to_csv(f"Places to visit in {zz}.csv ")

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
from bs4 import BeautifulSoup as BS
import re

In [ ]:
dfz = pd.read_csv(r"C:\Users\panka\OneDrive\Desktop\OverAll India\Bots\rem links links.csv")

In [ ]:
dfz

In [ ]:
url = "https://www.google.com/maps/@25.1297792,75.8448128,14z?entry=ttu"


driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

In [ ]:
for i in dfz["Links"][1002:]:
        
        driver.get(i)
        
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.presence_of_element_located((By.XPATH, '//h1[@class="DUwDvf lfPIob"]')))
            
        try:
            Name.append(driver.find_element(by="xpath", value ='//h1[@class="DUwDvf lfPIob"]').text)
        except NoSuchElementException:
            Name.append("None")

        try:
            Address.append(driver.find_element(by="xpath", value='//button[contains(@aria-label, "Address")]').get_attribute("aria-label").lstrip("Address:"))
        except NoSuchElementException:
            Address.append("None")

        try:
            Website.append(driver.find_element(by="xpath", value='//a[contains(@aria-label, "Website")]/div/div[2]/div[1]').text)
        except NoSuchElementException:
            Website.append("None")

        try:
            Phone_No.append(driver.find_element(by="xpath", value='//button[contains(@aria-label, "Phone")]').get_attribute("aria-label").lstrip("Phone:"))
        except NoSuchElementException:
            Phone_No.append("None")

        try:
            Category.append(driver.find_element(by="xpath", value='//button[contains(@jsaction,"category")]').text)
        except NoSuchElementException:
            Category.append("None")

        try:
            Rating.append(driver.find_element(by="xpath", value='//span[contains(@aria-label,"stars ") and contains(@class,"ceNzKf")]').get_attribute("aria-label").replace("stars", "").strip())
        except NoSuchElementException:
            Rating.append("None")

        try:
            No_of_Reviews.append(driver.find_element(by="xpath", value='//span[contains(@aria-label, "reviews")]').text.strip("()"))
        except NoSuchElementException:
            No_of_Reviews.append("None")

        linkk.append(driver.current_url)

        
    

driver.quit()
        

In [ ]:
dfz[dfz["Links"] == "https://support.google.com/maps/answer/3092445?entry=ttu&utm_source=maps&authuser=0&hl=en"].index

In [ ]:
df

In [ ]:
df.to_csv("remaining places 2.csv")

In [ ]:
#  wait = WebDriverWait(driver,10)
#     search_input = i
#     input_area = wait.until(EC.presence_of_element_located((By.XPATH, '//input[@id="searchboxinput"]')))
#     input_area.clear()
#     input_area.send_keys(search_input)

#     search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@id="searchbox-searchbutton"]')))
#     search_button.click()

#     time.sleep(3)

#     try:

#         time.sleep(1)

#         divSideBar=driver.find_element(By.CSS_SELECTOR,f"div[aria-label='Results for {search_input}']")
#         keepScrolling=True
#         while(keepScrolling):
#             divSideBar.send_keys(Keys.PAGE_DOWN)
#             time.sleep(0.5)
#             divSideBar.send_keys(Keys.PAGE_DOWN)
#             time.sleep(0.5)
#             html =driver.find_element(By.TAG_NAME, "html").get_attribute('outerHTML')
#             if(html.find("You've reached the end of the list.")!=-1):
#                 keepScrolling=False

#         list_data = driver.find_elements(By.XPATH, f'//div[@aria-label="Results for {search_input}"]/div')

#         Restaurants_Links = []

#         for j in list_data:
#             try:
#                 Restaurants_Links.append(j.find_element(by="xpath", value='.//div/a').get_attribute("href"))
#             except NoSuchElementException:
#                 Restaurants_Links.append("None")

#         cleaned_restaurants_Links = [k for k in Restaurants_Links if k != "None"]

#         df = pd.DataFrame()

#         df["Links"] = cleaned_restaurants_Links
#         df["Location"] = i.lstrip("Places to visit in ")

#         df.to_csv(f'{i.lstrip("Places to visit in ")} links.csv')


In [ ]:
# cleaned_restaurants_Links[28]

In [ ]:
# #Name of the Homestay :-

# driver.find_element(by="xpath", value ='//h1[@class="DUwDvf lfPIob"]').text

In [ ]:
# #Address :-
# driver.find_element(by="xpath", value='//button[@data-item-id="address"]/div/div[2]/div[1]').text

In [ ]:
# # Website :-
# driver.find_element(by="xpath", value='//a[contains(@aria-label, "Website")]/div/div[2]/div[1]').text

In [ ]:
# #Phone Number :-
# driver.find_element(by="xpath", value='//button[contains(@aria-label, "Phone")]/div/div[2]/div[1]').text

In [ ]:
# #overall ratings

# driver.find_element(by="xpath", value='//span[contains(@aria-label,"stars ") and contains(@class,"ceNzKf")]').get_attribute("aria-label")

In [ ]:
# driver.find_element(by="xpath", value='//span[contains(@aria-label, "reviews")]').text

In [ ]:
# hospital_data = []

In [ ]:
# for link in each_data_links[4:10]:
#     if link is not None:
#         # Visit the link
#         driver.get(link)

#         try:
#             # Extract data from the page
#             name = driver.find_element(By.XPATH, './/h1[@class="DUwDvf lfPIob"]').text
#             address = driver.find_element(By.XPATH, '(//div[@class="Io6YTe fontBodyMedium kR99db "])[1]').text
#             phone_number = driver.find_element(By.XPATH, '(//div[@class="Io6YTe fontBodyMedium kR99db "])[3]').text
#             num_reviews = driver.find_element(By.XPATH, '//div[@class="jANrlb"]/div[2]').get_attribute("aria-label")

#             # Store the extracted data in a dictionary
#             data = {
#                 "Name": name,
#                 "Address": address,
#                 "Phone Number": phone_number,
#                 "Total Reviews": num_reviews
#             }

#             # Append the data to the list
#             hospital_data.append(data)

#         except NoSuchElementException:
#             # Handle cases where the data is not found on the page
#             print(f"Data not found for link: {link}")

# # Print or process the hospital_data list as needed
# for data in hospital_data:
#     print(data)

In [ ]:
# divSideBar=driver.find_element(By.CSS_SELECTOR,f"div[aria-label='Results for {search_input}']")
# keepScrolling=True
# while(keepScrolling):
#     divSideBar.send_keys(Keys.PAGE_DOWN)
#     time.sleep(0.5)
#     divSideBar.send_keys(Keys.PAGE_DOWN)
#     time.sleep(0.5)
#     html =driver.find_element(By.TAG_NAME, "html").get_attribute('outerHTML')
#     if(html.find("You've reached the end of the list.")!=-1):
#         keepScrolling=False

In [ ]:
# bed_and_breakfast_link = []

# for i in list_data:
#     try:
#         bed_and_breakfast_link.append(i.find_element(by="xpath", value='.//div/a').get_attribute("href"))
#     except NoSuchElementException:
#         bed_and_breakfast_link.append("None")

# cleaned_bed_and_breakfast_link = [i for i in bed_and_breakfast_link if i != "None"]